# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [11]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

print(html)

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

<select class="movie_name" id="sbox_mid" name="movie_id">
<option value="">請選擇片名</option>
<option data-name="菠蘿蜜" value="10412">菠蘿蜜</option>
<option data-name="韓影印象影展" value="10410">韓影印象影展</option>
<option data-name="陰兒" value="10393">陰兒</option>
<option data-name="這一次不再錯過你" value="10389">這一次不再錯過你</option>
<option data-name="別告訴她" value="10382">別告訴她</option>
<option data-name="鬼屋實錄：惡魔之家" value="10378">鬼屋實錄：惡魔之家</option>
<option data-name="驅魔直播" value="10375">驅魔直播</option>
<option data-name="真愛裁會贏" value="10362">真愛裁會贏</option>
<option data-name="那一夜" value="10345">那一夜</option>
<option data-name="兔嘲男孩" value="10283">兔嘲男孩</option>
<option data-name="海上鋼琴師4K數位修復" value="10425">海上鋼琴師4K數位修復</option>
<option data-name="鹿皮奇談" value="10420">鹿皮奇談</option>
<option data-name="大約在冬季" value="10408">大約在冬季</option>
<option data-name="倒數計死" value="10390">倒數計死</option>
<option data-name="吹哨人" value="10381">吹哨人</option>
<option data-name="幸福試用期" value="10374">幸福試用期</option>
<option data-name="竊竊屍語" value

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [3]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10134

In [4]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 彰化, 代號(area_id): 22
放映地區: 南投, 代號(area_id): 13
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14
放映地區: 基隆, 代號(area_id): 18
放映地區: 宜蘭, 代號(area_id): 11
放映地區: 花蓮, 代號(area_id): 12
放映地區: 台東, 代號(area_id): 9
放映地區: 金門, 代號(area_id): 24
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [4]:
# 指定放映地區
area_id = 28

In [5]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

一月 3
一月 4
一月 5
一月 6
一月 7


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [8]:
# 選定要觀看的日期
date = "2020-01-07"

In [9]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [9]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
print(soup)

<html><body><div class="pc-movie-schedule-form">
<!-- 若今天沒有時刻表，顯示"本日電影未上映" -->
<p class="no_released">本日電影未上映</p>
<div class="booking_modal jq_booking_modal">
<div class="booking_modal_backdrop"></div>
<!-- 時報訂票 ＆ 有官網訂票 新增 class => no_seats -->
<div class="booking_modal_content jq_booking_modal_content">
<form action="https://movies.yahoo.com.tw/ticket/ezding_ticket" id="form_schedule" method="get" target="_blank">
<div class="ezding_ticket_value">
<input name="movie_id" type="hidden" value=""/>
<input name="cinema_id" type="hidden" value=""/>
<input name="session_id" type="hidden" value=""/>
<input name="date" type="hidden" value=""/>
<input name="time" type="hidden" value=""/>
<input name="campaign_code" type="hidden" value="yahoopc"/>
</div>
<ul class="top_infos">
<li>
<p>片名</p>
<h3 id="top_infos_movie"></h3>
</li>
<li>
<p>影城</p>
<h3 id="top_infos_theater"></h3>
</li>
<li>
<p>日期</p>
<h3 id="top_infos_date"></h3>
</li>
<li>
<p>版本</p>
<h3 id="top_infos_movie_type"></h3>
</li>
<li>
<p>